# Data Management Functionality

##  Introduction

Welcome to this Jupyter notebook, which is designed to showcase and explain the usage of functions defined in the src/data_management module. These functions have been specifically developed to streamline the process of handling and managing data, particularly focusing on the download and processing of .gz files from the sitemap of the WiWi faculty at Humboldt University of Berlin (HU Berlin).

## Objectives

In this notebook, we will:

1.	Introduce the key functions in the src/data_management module.
2.	Demonstrate how to use these functions effectively.
3.	Provide examples and use cases to illustrate their practical application.
4.	Explain the underlying logic and parameters of each function to ensure a clear understanding of their purpose and usage.

## Overview

The functions in src/data_management have been updated to enhance their capabilities, allowing for efficient downloading and processing of compressed .gz files. These updates are crucial for handling large datasets, ensuring that data extraction from the WiWi faculty’s sitemap is both effective and user-friendly.

We will begin by exploring the individual functions, followed by detailed demonstrations that will guide you through the process of utilizing these tools to manage and analyze data efficiently.

Please note, that this notebook will not showcase the upload of the derived files into the databases, as it is not the goal of this notebook.

In [1]:
import os
import pandas as pd
import sys

# Get the current working directory
cwd = os.getcwd()

# Add the '../scripts' directory to the system path
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../scripts')))
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../src')))

from data_management.i_scheme_downloader import *
from data_management.ii_unzip_convert_json import *
from data_management.iii_content_downloader import *
from data_management.iv_html_retriever_cleaner import *
from data_management.v_chunker_embedder import *


SITEMAP_URL = 'https://www.wiwi.hu-berlin.de/sitemap.xml.gz'
PATTERN = r'''<loc>(https://www\.wiwi\.hu-berlin\.de/en/(?!.*\.jpeg|.*\.pdf|.*\.png|.*\.jpg).*?)(?<!/view)</loc>\s*<lastmod>([^<]+)</lastmod>'''

/Users/s.konchakova/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 00:42:38,967 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2024-07-18 00:42:41,575 - INFO - Load pretrained SentenceTransformer: BAAI/bge-small-en
2024-07-18 00:42:43,695 - INFO - 2 prompts are loaded, with the keys: ['query', 'text']


## Download and Unpack the Sitemap File

In this step, we will download the sitemap file from the provided web link, which contains links to all pages of the website along with their last update dates. The sitemap file is compressed in .gz format, so we will also unpack it to access the data.

In [18]:
file = download_file(SITEMAP_URL)

content = unzip_local_file(file).decode('utf-8')

2024-07-18 00:54:58,426 - INFO - Unzipping local file: /tmp/sitemap.xml.gz
2024-07-18 00:54:58,433 - INFO - Successfully unzipped local file: /tmp/sitemap.xml.gz


In [19]:
print(f"The preview of the extracted content of the gz file: {content[:500]}")

The preview of the extracted content of the gz file: <?xml version="1.0" encoding="UTF-8"?>
<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.sitemaps.org/schemas/sitemap/0.9 http://www.sitemaps.org/schemas/sitemap/0.9/sitemap.xsd">
<url>
  <loc>https://www.wiwi.hu-berlin.de</loc>
  <lastmod>2021-11-09T20:03:57+01:00</lastmod>
  
  
</url>
<url>
  <loc>https://www.wiwi.hu-berlin.de/en/Professorships/vwl/microeconomics/people/wlefez</loc>
  <lastmod>2023-


As we 